# Fortigate Firewall Formatting Logs

Este script foi desenvolvido para extrair informações relevantes dos logs do webfilter de usuários do firewall FortiGate da Fortinet. O objetivo é fornecer uma maneira simples e eficiente de analisar o histórico de navegação de usuários, permitindo verificar sites visitados, se o acesso foi permitido ou bloqueado pelo firewall, detalhes da URL acessada, e outras informações de tráfego relacionadas ao web filter. Com isso, é possível realizar uma análise mais detalhada do comportamento de navegação dentro da rede, facilitando a monitoração e auditoria do tráfego de internet. Este script organiza os dados de maneira legível, ajudando a identificar padrões de acesso, sites bloqueados e permitir o acompanhamento eficaz da utilização da web pelos usuários monitorados.

Para usar o script através deste notebook atribua a variavel "input_file" o caminho do log do webfiltro do usuario que você deseja inspecionar

In [ ]:
import re

def informacao_relevante(log_linha):
    patterns = {
        'user': r'user="([^"]*)"',
        'hostname': r'hostname="([^"]*)"',
        'url': r'url="([^"]*)"',
        'action': r'action="([^"]*)"'
    }

    result = {}
    
    for key, pattern in patterns.items():
        match = re.search(pattern, log_linha)
        if match:
            result[key] = match.group(1)
        else:
            result[key] = 'N/A'
    
    return result

def main(input_file, output_file):
    hostname_count = {}  
    last_occurrence = {}  
    
    with open(input_file, 'r') as infile:
        for linha in infile:
            info = informacao_relevante(linha)
            hostname = info['hostname']
            
            if hostname in hostname_count:
                hostname_count[hostname] += 1
            else:
                hostname_count[hostname] = 1
                last_occurrence[hostname] = info
    
    # Ordenar os hostnames em ordem alfabética
    sorted_hostnames = sorted(hostname_count.keys())
    
    with open(output_file, 'w') as outfile:
        for hostname in sorted_hostnames:
            count = hostname_count[hostname]
            if count > 0:
                info = last_occurrence[hostname]
                formatted_linha = (f"user={info['user']} "
                                   f"hostname={hostname} action={info['action']} "
                                   f"url={info['url']} hostnamerepeat={count}\n")
                outfile.write(formatted_linha)

input_file = "forticloud-traffic-forward-2024-12-02_1031.log"  
output_file = "relatorio_de_firewall.log"  

# Chama a função principal
if __name__ == '__main__':
    main(input_file, output_file)